# KOL Data Standardization - Step 2 (Data Transformation)

In [1]:
"""
KOL Data Standardization - Step 2 (Load the Data from landing area and apply some transformations and business logic to it.)

This module represents the second step in KOL Data Standardization process where we load the landing area data and transform it using the following business rules:
- Merge the data of same KOLs (data points with same mdm_id) and ensure that the latest batch data is updated for all columns except degree and speciality.
- Since, a KOL can have multiple degrees and multiple specialities, concat these datapoints across the landing stage batches.
- Based on the master tables, apply name-id mappings to degree, speciality, and profile status.
- Handle null values on other columns
The above transformations will ensure data consistency and will allow us to prepare the staging area before loading it into the final reporting layer database.
"""

import pandas as pd
from IPython.display import display

In [2]:
# Master Tables Path - Used for business logic implementation

SPECIALITY_MASTER_PATH = "master_tables/speciality_master.csv"
PROFILE_STATUS_MATER_PATH = "master_tables/profile_status_master.csv"
DEGREE_MASTER_PATH = "master_tables/degree_master.csv"
LANDING_AREA_FILE_PATH = "output_store/landing_area.xlsx"
STAGING_AREA_FILE_PATH = "output_store/staging_area.xlsx"

In [3]:
def get_master_tables():
    """Utility Function to fetch all the master tables for the KOL data"""
    degree_master = pd.read_csv(DEGREE_MASTER_PATH)
    degree_master = degree_master.set_index("degree").to_dict()["id"]

    speciality_master = pd.read_csv(SPECIALITY_MASTER_PATH)
    speciality_master = speciality_master.set_index("speciality").to_dict()["id"]

    profile_master = pd.read_csv(PROFILE_STATUS_MATER_PATH)
    profile_master = profile_master.set_index("id").to_dict()["profile_status"]

    return degree_master, speciality_master, profile_master


def speciality_mapper(speciality_data, speciality_master):
    """Utility Function to map specialities of a KOL with their respective ids"""
    speciality_data = str(speciality_data).strip()
    if len(speciality_data) != 0:
        speciality_ids = [
            str(speciality_master.get(x.strip())) for x in speciality_data.split(",")
        ]
        return ",".join(sorted(speciality_ids))
    return "0"


def degree_mapper(degree_data, degree_master):
    """Utility Function to map degrees of a KOL with their respective ids"""
    degree_data = str(degree_data).strip()
    if len(degree_data) != 0:
        degree_ids = [str(degree_master.get(x.strip())) for x in degree_data.split(",")]
        return ",".join(sorted(degree_ids))
    return "0"


def prepare_staging_area():
    """Main Data Transformation function to transform data and make it ready for the staging area."""
    print("TASK: Loading the Landing Area Dataframe")
    landing_df = pd.read_excel(LANDING_AREA_FILE_PATH)
    landing_df = landing_df.sort_values(["mdm_id", "batch_id"]).reset_index(drop=True)
    print("TASK: Successfully loaded the Landing Area Dataframe")
    display(landing_df.head())

    # Merging Data of the same KOL (same mdm_id) based on batches and below business logic
    print(
        "\nTASK (Transformation Process 1): Merging the data from different batches in landing area for the same KOLs based on some business logic."
    )
    staging_df = (
        landing_df.groupby("mdm_id")
        .agg(
            {
                "first_name": "last",  # take the last non-null value among the batches
                "last_name": "last",  # take the last non-null value among the batches
                "age": "last",  # take the last non-null value among the batches
                "city": "last",  # take the last non-null value among the batches
                "state": "last",  # take the last non-null value among the batches
                "profile_status": "last",  # take the last non-null value among the batches
                "speciality": lambda x: ", ".join(
                    x.dropna().unique()
                ),  # combine unique specialities among the batches
                "degree": lambda x: ", ".join(
                    x.dropna().unique()
                ),  # combine unique degrees among the batches
                "batch_id": "last",  # take the latest batch_id
            }
        )
        .reset_index()
    )
    staging_df[["last_name", "city", "state"]] = staging_df[
        ["last_name", "city", "state"]
    ].fillna("-")
    staging_df["age"] = staging_df["age"].fillna(-1).astype("int")
    print("TASK: Succcessfully implemented the merging logic")
    display(staging_df.head())

    print(
        "\nTASK (Transformation Process 2): Mapping Specialities, Degrees, and Profiling status of various KOLs based on master tables"
    )
    degree_master, speciality_master, profile_master = get_master_tables()
    staging_df["speciality"] = staging_df["speciality"].apply(
        lambda x: speciality_mapper(x, speciality_master)
    )
    staging_df["degree"] = staging_df["degree"].apply(
        lambda x: degree_mapper(x, degree_master)
    )
    staging_df["profile_status"] = (
        staging_df["profile_status"].fillna(0).astype("int").apply(profile_master.get)
    )
    print("TASK: Succcessfully prepared the Staging area dataframe")
    display(staging_df.head())

    return staging_df

In [4]:
if __name__ == "__main__":
    print("Starting the Data Transformation Process\n\n")
    staging_df = prepare_staging_area()
    staging_df.to_excel(STAGING_AREA_FILE_PATH, index=False)
    print(f"\n\nSuccessfully updated the staging area: {STAGING_AREA_FILE_PATH}")

Starting the Data Transformation Process


TASK: Loading the Landing Area Dataframe
TASK: Successfully loaded the Landing Area Dataframe


,mdm_id,first_name,last_name,age,city,state,profile_status,speciality,degree,batch_id
0,109,Justin,Davenport,NaN,Laurenport,Washington,NaN,Dermatopharmacology,"PHD,BHMS,MD",2024-09-07 14:43:31.929
1,119,Jared,NaN,43.0,NaN,North Dakota,NaN,"Forensic Pathology,Pruritus,Emergency Medicine...",MD,2024-09-07 14:43:31.966
2,124,Justin,Sexton,NaN,Bethstad,Colorado,NaN,"Immunological Disorders,Interventional Pain Me...",NaN,2024-09-07 14:43:31.929
3,130,Courtney,NaN,NaN,Jamesview,Montana,NaN,NaN,NaN,2024-09-07 14:43:31.966
4,133,Alex,NaN,19.0,Thomasside,NaN,1.0,"Eosinophilic Esophagitis,Public Health And Gen...",MS,2024-09-07 14:43:31.929



TASK (Transformation Process 1): Merging the data from different batches in landing area for the same KOLs based on some business logic.
TASK: Succcessfully implemented the merging logic


,mdm_id,first_name,last_name,age,city,state,profile_status,speciality,degree,batch_id
0,109,Justin,Davenport,-1,Laurenport,Washington,NaN,Dermatopharmacology,"PHD,BHMS,MD",2024-09-07 14:43:31.929
1,119,Jared,-,43,-,North Dakota,NaN,"Forensic Pathology,Pruritus,Emergency Medicine...",MD,2024-09-07 14:43:31.966
2,124,Justin,Sexton,-1,Bethstad,Colorado,NaN,"Immunological Disorders,Interventional Pain Me...",,2024-09-07 14:43:31.929
3,130,Courtney,-,-1,Jamesview,Montana,NaN,,,2024-09-07 14:43:31.966
4,133,Alex,-,19,Thomasside,-,1.0,"Eosinophilic Esophagitis,Public Health And Gen...",MS,2024-09-07 14:43:31.929



TASK (Transformation Process 2): Mapping Specialities, Degrees, and Profiling status of various KOLs based on master tables
TASK: Succcessfully prepared the Staging area dataframe


,mdm_id,first_name,last_name,age,city,state,profile_status,speciality,degree,batch_id
0,109,Justin,Davenport,-1,Laurenport,Washington,Not Profiled,295,"2,3,6",2024-09-07 14:43:31.929
1,119,Jared,-,43,-,North Dakota,Not Profiled,"106,187,383,9",2,2024-09-07 14:43:31.966
2,124,Justin,Sexton,-1,Bethstad,Colorado,Not Profiled,"330,466",0,2024-09-07 14:43:31.929
3,130,Courtney,-,-1,Jamesview,Montana,Not Profiled,0,0,2024-09-07 14:43:31.966
4,133,Alex,-,19,Thomasside,-,Partially Profiled,"307,70",8,2024-09-07 14:43:31.929




Successfully updated the staging area: output_store/staging_area.xlsx
